In [1]:
import pandas as pd
import numpy as np
from bs4 import *

In [2]:
import os    
train = pd.read_csv('labeledTrainData.tsv', header=0, delimiter="\t", quoting=3)
pre = pd.read_csv('testData.tsv', header=0, delimiter="\t", quoting=3)

In [3]:
#preprocess file 
label = train['sentiment']
train_data = []
pre_data = []
for i in range(len(train['review'])):
    train_data.append(BeautifulSoup(train['review'][i], "html.parser").get_text())
test_data = []
for i in range(len(pre['review'])):
    pre_data.append(BeautifulSoup(pre['review'][i], "html.parser").get_text())

In [4]:
print(len(train_data))
print(len(pre_data))

25000
25000


In [5]:
print(train_data[0])

"With all this stuff going down at the moment with MJ i've started listening to his music, watching the odd documentary here and there, watched The Wiz and watched Moonwalker again. Maybe i just want to get a certain insight into this guy who i thought was really cool in the eighties just to maybe make up my mind whether he is guilty or innocent. Moonwalker is part biography, part feature film which i remember going to see at the cinema when it was originally released. Some of it has subtle messages about MJ's feeling towards the press and also the obvious message of drugs are bad m'kay.Visually impressive but of course this is all about Michael Jackson so unless you remotely like MJ in anyway then you are going to hate this and find it boring. Some may call MJ an egotist for consenting to the making of this movie BUT MJ and most of his fans would say that he made it for the fans which if true is really nice of him.The actual feature film bit when it finally starts is only on for 20 mi

In [6]:
# 合并训练和测试集以便进行TFIDF向量化操作
data_all = train_data + pre_data
len_train = len(train_data)
print(len_train)
print(train_data)

25000


IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.

Current values:
NotebookApp.iopub_data_rate_limit=1000000.0 (bytes/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [7]:
from nltk.corpus import stopwords
#英文停止词，set()集合函数消除重复项
list_stopWords = list(set(stopwords.words('english')))
print(list_stopWords)

['from', 'them', 'have', 'on', 'they', 'am', 'up', 'aren', 'wouldn', 'didn', 'while', 'few', 'own', "wasn't", 'it', 'itself', "should've", 'most', "you'll", 'hasn', 'before', 'myself', 'wasn', 'hadn', 'doesn', 'there', 'couldn', 'his', 'he', 'off', 's', "needn't", 'did', 'are', "you'd", 'nor', 'is', 'once', 'yourselves', 'o', "aren't", 'how', 'who', 'out', 'further', 'or', 'more', 'yours', 'because', "shan't", 'i', 'me', 'about', 'under', 'against', 'm', 'themselves', 'down', "hasn't", 'over', 'where', 'herself', 'had', 'has', 'as', 'to', 'shan', 'until', 'below', 'y', "haven't", 'some', 'our', "that'll", 'but', 'which', 'why', "mightn't", 'does', 'should', 'now', 'above', 'such', "shouldn't", 'with', 'than', 'a', 'haven', 'here', "you're", "didn't", 'all', 'having', 'my', 'when', 'so', 'weren', 'in', 'himself', 'hers', 'mustn', 'shouldn', 'this', 'during', 'any', "couldn't", 'been', 'each', 'very', "don't", 'its', 'only', 'their', "weren't", 'ain', 'will', "mustn't", 'she', 'him', 'by

In [8]:
from gensim import corpora

# bow 模型    
import re
#lower后，去除非 azAZ开头的单词
# [[word,word,........]]
texts = [[word for word in re.sub("[^a-zA-Z]", " ", doc.lower()).split() if word != "" and word not in list_stopWords] for doc in data_all]
"""
token2id : dict of (str, int)
    token -> tokenId.
id2token : dict of (int, str)
"""
dictionary = corpora.Dictionary(texts)

#corpus [ [(wordid:frequency),] ]
corpus = [dictionary.doc2bow(text) for text in texts]


In [9]:
len(dictionary.id2token)

0

In [10]:
from gensim import models
tfidf_model = models.TfidfModel(corpus=corpus, id2word=dictionary, normalize=True) 
#把所有的文档转化成tf-idf 
#转化后的文档是一个稀疏矩阵，这时采用密集矩阵来表示的话，会造成极大的内存浪费，
#所以gensim内部是用稀疏矩阵的形式来表示
corpus_tfidf = tfidf_model[corpus]

In [11]:
#每个句子长度不一，
#倘若使用密集矩阵的话是一样的长度
len8 = len(corpus_tfidf[8])#
len9 = len(corpus_tfidf[9])
print("len8 :{0} -- len9 :{1} ".format(len8,len9))

len8 :81 -- len9 :22 


In [12]:
## lsi 主题模型，作为特征向量
lsi_model = models.LsiModel(corpus_tfidf, id2word=dictionary, num_topics=200)
corpus_lsi = lsi_model[corpus_tfidf]

# 提取数字，转化为numpy的矩阵
all_x = [[v for k,v in doc] for doc in corpus_lsi]

In [13]:
len(corpus_lsi[7])


200

In [42]:
# print(np.shape(corpus_lsi))
# (50000, 200, 2)
#三个主题每个主题用一个word
print(lsi_model.print_topics(3,1))

#lsi_model.print_topics(2)
# print(corpus_lsi[0])

# import numpy as np
# print(np.shape(all_x))

[(0, '0.163*"movie"'), (1, '-0.302*"bad"'), (2, '-0.665*"show"')]


In [22]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score

knn_model = KNeighborsClassifier(n_neighbors=5)
knn_model.fit(all_x[:len_train], label)

scores = cross_val_score(knn_model, all_x[:len_train], label, cv=10, scoring='roc_auc')
print("\nknn 10折交叉验证得分: \n", scores)
print("\nknn 10折交叉验证平均得分: ", np.mean(scores))


knn算法 10折交叉验证得分: 
 [0.82179872 0.81945952 0.83272512 0.82310144 0.829504   0.82556576
 0.81654688 0.82681824 0.8256928  0.81336256]

knn算法 10折交叉验证平均得分: 
 0.823457504


In [43]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_val_score

lr_model = LogisticRegression(C=0.1, max_iter=100)
lr_model.fit(all_x[:len_train], label)

scores = cross_val_score(lr_model, all_x[:len_train], label, cv=10, scoring='roc_auc')
print("\n logistic regression 10折交叉验证得分: \n", scores)
print("\n logistic regression 10折交叉验证平均得分: \n", np.mean(scores))


 logistic regression 10折交叉验证得分: 
 [0.93431744 0.9281536  0.9413472  0.93139584 0.93204096 0.93203136
 0.9335392  0.94093824 0.9232384  0.9319104 ]

 logistic regression 10折交叉验证平均得分: 
 0.932891264


In [44]:
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
rf_model = RandomForestClassifier(n_estimators=100, max_depth=8, random_state=0)
rf_model.fit(all_x[:len_train], label)

scores = cross_val_score(rf_model, all_x[:len_train], label, cv=10, scoring='roc_auc')
print("\n RandomForest 10折交叉验证得分: \n", scores)
print("\n RandomForest 10折交叉验证平均得分: \n", np.mean(scores))


 RandomForest 10折交叉验证得分: 
 [0.90736832 0.9032128  0.91823424 0.90634624 0.90518336 0.9027776
 0.90534528 0.90977024 0.89864448 0.90629312]

 RandomForest 10折交叉验证平均得分: 
 0.9063175680000001


In [16]:

from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
import numpy as np

xgb_model = XGBClassifier(n_estimators=150, min_samples_leaf=3, max_depth=6)
"""
AttributeError: 'list' object has no attribute 'shape'
list => np.array
"""
xgb_model.fit(np.array(all_x[:len_train]), label)

scores = cross_val_score(xgb_model, np.array(all_x[:len_train]), label, cv=10, scoring='roc_auc')
print("\nXGB 10折交叉验证得分: \n", scores)
print("\nXGB 10折交叉验证平均得分: \n", np.mean(scores))


XGB 10折交叉验证得分: 
 [0.93912832 0.93999936 0.94611776 0.93853504 0.93499392 0.93230208
 0.93981824 0.94093696 0.92940416 0.93759104]

XGB 10折交叉验证平均得分: 
 0.9378826879999999


In [14]:

import xgboost as xgb